<a href="https://colab.research.google.com/github/witheunjin/TF-models_ejlee/blob/master/Taobao_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://programs.team/analysis-of-taobao-user-behavior-data.html


In [1]:
import time
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [2]:
raw_sample = pd.read_csv('drive/MyDrive/Dataset/Taobao/raw_sample.csv')
ad_feature = pd.read_csv('drive/MyDrive/Dataset/Taobao/ad_feature.csv')
user_profile = pd.read_csv('drive/MyDrive/Dataset/Taobao/user_profile.csv')

## Raw Sample

In [3]:
# print 5 rows
raw_sample.head()

,user,time_stamp,adgroup_id,pid,nonclk,clk
0,581738,1494137644,1,430548_1007,1,0
1,449818,1494638778,3,430548_1007,1,0
2,914836,1494650879,4,430548_1007,1,0
3,914836,1494651029,5,430548_1007,1,0
4,399907,1494302958,8,430548_1007,1,0


In [4]:
raw_sample.count()

user          26557961
time_stamp    26557961
adgroup_id    26557961
pid           26557961
nonclk        26557961
clk           26557961
dtype: int64

In [5]:
# print summary 
raw_sample.describe()

,user,time_stamp,adgroup_id,nonclk,clk
count,2.655796e+07,2.655796e+07,2.655796e+07,2.655796e+07,2.655796e+07
mean,5.682055e+05,1.494355e+09,5.130175e+05,9.485632e-01,5.143678e-02
std,3.297502e+05,1.987553e+05,2.183782e+05,2.208869e-01,2.208869e-01
min,1.000000e+00,1.494000e+09,1.000000e+00,0.000000e+00,0.000000e+00
25%,2.829190e+05,1.494171e+09,3.606780e+05,1.000000e+00,0.000000e+00
50%,5.649520e+05,1.494345e+09,5.638330e+05,1.000000e+00,0.000000e+00
75%,8.521910e+05,1.494516e+09,6.867050e+05,1.000000e+00,0.000000e+00
max,1.141729e+06,1.494691e+09,8.468110e+05,1.000000e+00,1.000000e+00


In [6]:
# average items per user
item_per_user = raw_sample.groupby(['user'])['adgroup_id'].count()
print(item_per_user)
item_per_user.describe()

user
1           4
2           3
3          15
4          74
5           3
           ..
1141725    21
1141726    49
1141727     9
1141728     1
1141729    23
Name: adgroup_id, Length: 1141729, dtype: int64


count    1.141729e+06
mean     2.326118e+01
std      6.539253e+01
min      1.000000e+00
25%      3.000000e+00
50%      7.000000e+00
75%      2.000000e+01
max      3.756000e+03
Name: adgroup_id, dtype: float64

In [17]:
item_list_per_user = raw_sample.groupby('user')['adgroup_id'].count()
print('The number of items per user (just 1000 row)')
print(list(item_list_per_user.head(10000)))
print('\nAverage number of item per user')
print(item_list_per_user.mean())
print(item_list_per_user.max())
#for key, item in item_list_per_user:
#  if key < 10:
#    print(item_list_per_user.get_group(key), "\n\n")

The number of items per user (just 1000 row)
[4, 3, 15, 74, 3, 11, 665, 12, 3, 3, 6, 3, 9, 15, 27, 4, 2, 37, 73, 3, 3, 3, 3, 12, 19, 3, 7, 3, 4, 15, 3, 6, 68, 1, 6, 3, 370, 3, 3, 3, 10, 23, 13, 10, 6, 57, 18, 3, 9, 99, 51, 2, 6, 3, 22, 12, 41, 3, 3, 3, 6, 372, 3, 27, 15, 3, 183, 6, 35, 5, 5, 1, 36, 53, 15, 66, 3, 12, 2, 6, 7, 4, 3, 3, 3, 3, 3, 6, 48, 9, 3, 6, 17, 9, 19, 27, 20, 12, 3, 15, 128, 9, 21, 6, 11, 12, 6, 21, 6, 3, 3, 3, 9, 1, 1, 9, 27, 6, 3, 23, 4, 182, 18, 165, 80, 33, 10, 12, 6, 5, 3, 3, 22, 60, 32, 9, 49, 15, 3, 1, 21, 58, 47, 5, 3, 9, 18, 2, 9, 47, 25, 43, 3, 6, 32, 44, 109, 4, 6, 15, 3, 33, 48, 6, 7, 3, 70, 3, 6, 3, 3, 9, 21, 3, 15, 14, 5, 2, 3, 3, 33, 8, 12, 68, 3, 6, 21, 9, 76, 9, 3, 29, 5, 6, 6, 199, 18, 6, 3, 9, 11, 7, 3, 18, 6, 3, 6, 6, 6, 10, 49, 6, 27, 3, 9, 33, 12, 9, 2, 12, 9, 25, 27, 23, 3, 44, 3, 55, 64, 7, 29, 6, 4, 75, 3, 11, 40, 9, 2, 54, 35, 15, 10, 15, 5, 3, 33, 3, 13, 3, 2, 13, 34, 15, 6, 17, 15, 24, 3, 68, 77, 3, 279, 95, 3, 18, 20, 4, 18, 6, 42, 1, 9, 

In [8]:
user_per_item = raw_sample.groupby(['adgroup_id']).count()
print(user_per_item)
user_per_item.head()
user_per_item.describe()

            user  time_stamp  pid  nonclk  clk
adgroup_id                                    
1              1           1    1       1    1
2              6           6    6       6    6
3              1           1    1       1    1
4              1           1    1       1    1
5              1           1    1       1    1
...          ...         ...  ...     ...  ...
846807         5           5    5       5    5
846808         2           2    2       2    2
846809         2           2    2       2    2
846810         1           1    1       1    1
846811         1           1    1       1    1

[846811 rows x 5 columns]


,user,time_stamp,pid,nonclk,clk
count,846811.000000,846811.000000,846811.000000,846811.000000,846811.000000
mean,31.362324,31.362324,31.362324,31.362324,31.362324
std,200.434141,200.434141,200.434141,200.434141,200.434141
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,4.000000,4.000000,4.000000,4.000000,4.000000
75%,15.000000,15.000000,15.000000,15.000000,15.000000
max,70929.000000,70929.000000,70929.000000,70929.000000,70929.000000


In [9]:
print(raw_sample.pid.unique())

['430548_1007' '430539_1007']


## Ad Feature

In [10]:
ad_feature.count()

adgroup_id     846811
cate_id        846811
campaign_id    846811
customer       846811
brand          600481
price          846811
dtype: int64

In [11]:
ad_feature.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price
0,63133,6406,83237,1,95471.0,170.00
1,313401,6406,83237,1,87331.0,199.00
2,248909,392,83237,1,32233.0,38.00
3,208458,392,83237,1,174374.0,139.00
4,110847,7211,135256,2,145952.0,32.99


In [12]:
num_ctgry_per_user = ad_feature.groupby('customer')['cate_id'].count()
print("The number of category per each user")
print(list(num_ctgry_per_user))
print("\nThe average number of category per each user")
print(num_ctgry_per_user.mean())
print(num_ctgry_per_user.max())

The number of category per each user
[4, 1, 14, 4, 2, 2, 3, 5, 2, 3, 1, 2, 4, 15, 1, 2, 1, 1, 4, 1, 2, 1, 3, 3, 1, 30, 2, 14, 1, 1, 1, 1, 5, 7, 2, 1, 1, 14, 1, 19, 1, 5, 1, 3, 15, 9, 4, 2, 1, 9, 1, 1, 1, 3, 1, 1, 3, 4, 4, 4, 1, 5, 3, 1, 6, 1, 5, 1, 3, 1, 1, 6, 3, 7, 1, 7, 4, 1, 4, 8, 1, 1, 16, 2, 3, 1, 3, 1, 1, 54, 2, 1, 1, 1, 1, 4, 1, 1, 3, 2, 1, 2, 3, 15, 1, 3, 3, 1, 3, 1, 1, 19, 1, 5, 1, 4, 9, 1, 5, 1, 12, 5, 5, 1, 1, 3, 2, 5, 3, 31, 1, 1, 1, 5, 5, 2, 5, 26, 2, 1, 1, 1, 1, 1, 4, 2, 64, 1, 1, 2, 2, 19, 1, 1, 1, 4, 1, 8, 2, 1, 3, 1, 3, 1, 14, 1, 5, 1, 8, 1, 1, 1, 3, 2, 2, 4, 3, 1, 1, 1, 9, 5, 8, 2, 1, 3, 3, 1, 7, 1, 2, 2, 1, 2, 1, 1, 1, 11, 1, 2, 18, 2, 1, 2, 1, 23, 1, 2, 7, 2, 5, 4, 7, 2, 4, 2, 1, 1, 3, 1, 1, 1, 1, 1, 55, 2, 1, 2, 1, 1, 1, 3, 1, 2, 2, 31, 10, 1, 2, 3, 12, 7, 3, 1, 1, 13, 9, 4, 2, 1, 2, 4, 1, 2, 5, 5, 1, 2, 2, 2, 5, 1, 1, 3, 1, 8, 1, 3, 3, 1, 1, 2, 3, 1, 2, 3, 9, 3, 9, 4, 10, 4, 1, 7, 1, 1, 8, 1, 2, 3, 2, 1, 8, 1, 1, 1, 1, 1, 6, 2, 1, 1, 5, 3, 1, 2, 1, 2, 2, 1, 1, 16,

In [13]:
ad_feature.describe()

,adgroup_id,cate_id,campaign_id,customer,brand,price
count,846811.000000,846811.000000,846811.000000,846811.000000,600481.000000,8.468110e+05
mean,423406.000000,5868.593464,206552.604280,113180.406006,229254.422596,1.838867e+03
std,244453.423739,2705.171203,125192.340908,73435.834950,132288.849633,3.108877e+05
min,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000e-02
25%,211703.500000,4383.000000,97583.000000,47960.000000,110616.000000,4.900000e+01
50%,423406.000000,6183.000000,200780.000000,107278.000000,234423.000000,1.390000e+02
75%,635108.500000,7047.000000,314565.500000,172013.500000,344818.000000,3.520000e+02
max,846811.000000,12960.000000,423436.000000,255875.000000,461497.000000,1.000000e+08


## User Profile

In [14]:
user_profile.count()

userid                   1061768
cms_segid                1061768
cms_group_id             1061768
final_gender_code        1061768
age_level                1061768
pvalue_level              485851
shopping_level           1061768
occupation               1061768
new_user_class_level      716848
dtype: int64

In [15]:
user_profile.head()

,userid,cms_segid,cms_group_id,final_gender_code,age_level,pvalue_level,shopping_level,occupation,new_user_class_level
0,234,0,5,2,5,NaN,3,0,3.0
1,523,5,2,2,2,1.0,3,1,2.0
2,612,0,8,1,2,2.0,3,0,NaN
3,1670,0,4,2,4,NaN,1,0,NaN
4,2545,0,10,1,4,NaN,3,0,NaN


In [16]:
user_profile.describe()

,userid,cms_segid,cms_group_id,final_gender_code,age_level,pvalue_level,shopping_level,occupation,new_user_class_level
count,1.061768e+06,1.061768e+06,1.061768e+06,1.061768e+06,1.061768e+06,485851.000000,1.061768e+06,1.061768e+06,716848.000000
mean,5.713104e+05,1.512998e+01,5.542412e+00,1.644445e+00,3.410641e+00,1.759850,2.716854e+00,5.623074e-02,2.516379
std,3.295021e+05,2.535503e+01,3.178516e+00,4.786814e-01,1.227458e+00,0.581303,5.861036e-01,2.303669e-01,0.928334
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000,1.000000e+00,0.000000e+00,1.000000
25%,2.862458e+05,0.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,1.000000,3.000000e+00,0.000000e+00,2.000000
50%,5.714555e+05,0.000000e+00,4.000000e+00,2.000000e+00,3.000000e+00,2.000000,3.000000e+00,0.000000e+00,2.000000
75%,8.546662e+05,2.000000e+01,9.000000e+00,2.000000e+00,4.000000e+00,2.000000,3.000000e+00,0.000000e+00,3.000000
max,1.141729e+06,9.600000e+01,1.200000e+01,2.000000e+00,6.000000e+00,3.000000,3.000000e+00,1.000000e+00,4.000000
